# Multi-Armed Bandits

The most important feature distinguishing reinforcement learning from other types of  
learning is that it uses training information that evaluates the actions taken rather  
than instructs by giving correct actions.

There's multiple type of feedbacks:
- Purely evaluative feedback indicates how good the action taken was, but not whether it was the best or the worst action possible.
- Purely instructive feedback, on the other hand, indicates the correct action to take, independently of the action actually taken.

In this chapter we study the evaluative aspect of reinforcement learning in a simplified  
setting, one that does not involve learning to act in more than one situation. This  
nonassociative setting is the one in which most prior work involving evaluative feedback  
has been done, and it avoids much of the complexity of the full reinforcement learning  
problem.

The particular nonassociative, evaluative feedback problem that we explore is a simple  
version of the k-armed bandit problem.

## 2.1 A k-armed Bandit Problem

### What is k-armed bandit problem

Consider the following learning problem. You are faced repeatedly with a choice among  
k different options, or actions. After each choice you receive a numerical reward chosen  
from a stationary probability distribution that depends on the action you selected.  
Your objective is to maximize the expected total reward over some time period, for example,  
over 1000 action selections, or time steps.

In our **k-armed bandit problem**, each of the k actions has an expected or mean reward  
given that that action is selected; let us call this the **value** of that action.  
We denote the action selected on time step t as $A_t$, and the corresponding reward as $R_t$.  
The value then of an arbitrary action $a$, denoted $q_*(a)$, is the expected reward given that a is selected:

$q_*(a) = E[R_t | A_t = a]$

If you knew the value of each action, then it would be trivial to solve the k-armed bandit  
problem: you would always select the action with highest value. We assume that you do  
not know the action values with certainty, although you may have estimates. We denote  
the estimated value of action $a$ at time step $t$ as $Q_t(a)$.
We would like $Q_t(a)$ to be close to $q_*(a)$.

If you maintain estimates of the **action values**, then at any time step there is at least  
one action whose estimated value is greatest. We call these the **greedy actions**. When you  
select one of these actions, we say that you are **exploiting** your current knowledge of the  
values of the actions. If instead you select one of the **nongreedy actions**, then we say you  
are **exploring**, because this enables you to improve your estimate of the nongreedy action’s  
value

### K-armed bandit problem: Implementation

#### Numpy

[NumPy](https://numpy.org/) is the fundamental package for scientific computing in Python.  
It is a Python library that provides a multidimensional array object, various derived objects (such as masked arrays and matrices),  
and an assortment of routines for fast operations on arrays, including mathematical, logical,  
shape manipulation, sorting, selecting, I/O, discrete Fourier transforms, basic linear algebra, basic statistical operations, random simulation and much more.

#### Gymnasium

To create our environement k-armed bandit, we're gonne use gym,  
which is kind of the standard to create reproductible reinforcement learning environments.

[Gym](https://www.gymlibrary.dev/index.html) is an open source Python library for developing and comparing reinforcement learning algorithms  
by providing a standard API to communicate between learning algorithms and environments,  
as well as a standard set of environments compliant with that API.  
Since its release, Gym's API has become the field standard for doing this.

[Gymnasium](https://gymnasium.farama.org/) is a maintained fork of OpenAI’s Gym library.  
The Gymnasium interface is simple, pythonic, and capable of representing general RL problems,  
and has a compatibility wrapper for old Gym environments.

### K-armed Bandit Gymnasium Environment

In [ ]:
import numpy as np
import gymnasium as gym

from numba import int32, float64
from numba.experimental import jitclass

class KArmedBandit(gym.Env):

    def __init__(self, nb_arms=10, nb_steps=1000, mean=0, variance=1, noise_variance=1):
        self._nb_arms = nb_arms
        self._nb_steps = nb_steps

        self._mean = 0
        self._noise_mean = 0
        self._variance = variance
        self._noise_variance = noise_variance

        self.action_space = gym.spaces.Discrete(nb_arms)
        self.observation_space = gym.spaces.Discrete(1)
    
    def step(self, action):
        self._step += 1
    
        reward = self._arms[action]
        reward_noise = self.np_random.normal(self._noise_mean, self._noise_variance)
        terminated = self._step >= self._nb_steps

        info = { "is_optimal_action": int(action == np.argmax(self._arms)) }
        reward = reward + reward_noise

        return None, reward, terminated, False, info

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)

        self._step = 0
        self._arms = self.np_random.normal(self._mean, self._variance, size=self._nb_arms)

        return None, {}

plop()

## 2.2 Action-value Methods

We begin by looking more closely at methods for estimating the values of actions and  
for using the estimates to make action selection decisions, which we collectively call **action-value methods**.  
Recall that the true value of an action is the mean reward when that action is selected.  
One natural way to estimate this is by averaging the rewards actually received:  

$
\begin{aligned}
Q_t(a) &= {sum-of-rewards-when-a-taken-prior-to-t \over number-of-times-a-taken-prior-to-t} \\
       &= {\sum_{i=1}^{t-1} R_i · 1_{A_i = a} \over \sum_{i=1}^{t-1} 1_{A_i=a}}
\end{aligned}
$

where predicate denotes the random variable that is 1 if predicate is true and 0 if it is not.  
If the denominator is zero, then we instead define $Q_t(a)$ as some default value, such as 0.  
As the denominator goes to infinity, by the law of large numbers, $Q_t(a)$ converges to  
$q_*(a)$. We call this the sample-average method for estimating action values because each  
estimate is an average of the sample of relevant rewards.

The simplest action selection rule is to select one of the actions with the highest  
estimated value, that is, one of the greedy actions as defined in the previous section.  
If there is more than one greedy action, then a selection is made among them in some  
arbitrary way, perhaps randomly. We write this greedy action selection method as  

$
A_{t} .= argmax_a Q_t(a)
$ 

where argmax a denotes the action a for which the expression that follows is maximized (with ties broken arbitrarily).  
Greedy action selection always exploits current knowledge to maximize immediate reward;  
it spends no time at all sampling apparently inferior actions to see if they might really be better.  
A simple alternative is to behave greedily most of the time, but every once in a while,  
say with small probability $\epsilon$, instead select randomly from among all the actions with equal probability,  
independently of the action-value estimates.

We call methods using this near-greedy action selection rule **$\epsilon$-greedy methods**.  
An advantage of these methods is that, in the limit as the number of steps increases,  
every action will be sampled an infinite number of times, thus ensuring that all the $Q_t(a)$ converge to $q_*(a)$.  
This of course implies that the probability of selecting the optimal action converges to greater than $1-\epsilon$, that is, to near certainty.  

These are just asymptotic guarantees, however, and say little about the practical effectiveness of the methods.

#### Epsilon Greedy: Implementation

In [ ]:
from numba import int32, float64
from numba.experimental import jitclass

# Here we use numba to accelerate our code
# You can just ignore the jitclass part it doesn't impact the code logic
@jitclass([
    ('nb_actions', int32),
    ('epsilon', float64),
    ('q', float64[:]),
    ('reward_sum', float64[:]),
    ('nb_action_taken', float64[:]),
])
class EpsilonGreedy():

    def __init__(self, nb_actions, epsilon):
        self.nb_actions = nb_actions
        self.epsilon = epsilon

        self.q = np.zeros(self.nb_actions)
        self.reward_sum = np.zeros(self.nb_actions)
        self.nb_action_taken = np.zeros(self.nb_actions)

    def action(self):
        take_random_action_prob = np.random.uniform(0, 1)

        if take_random_action_prob < self.epsilon:
            return np.random.randint(0, self.nb_actions)
        else:
            return np.argmax(self.q)
    
    def observe(self, action, reward):
        self.nb_action_taken[action] += 1
        self.reward_sum[action] += reward

        self.q[action] = self.reward_sum[action] / self.nb_action_taken[action]
    
    def reset(self):
        self.q = np.zeros(self.nb_actions)
        self.reward_sum = np.zeros(self.nb_actions)
        self.nb_action_taken = np.zeros(self.nb_actions)

## 2.3 The 10-armed Testbed

To roughly assess the relative effectiveness of the greedy and $\epsilon$-greedy action-value methods,  
we compared them numerically on a suite of test problems.  
This was a set of 2000 randomly generated k-armed bandit problems with $k=10$.

In [ ]:
%run -i ../../tools/armed_bandits.py

nb_arms  = 10   # k = 10
nb_steps = 2000 # Set of 2000 randomly generated k-armed bandit problems

# Create the gym k-armed-bandit environement
env = KArmedBandit(nb_arms=nb_arms, nb_steps=nb_steps)

# Plot sample distribution
plot_env_sample(env)

### Tools to run experiments

To be able to run experiments we will need three functions:

1. The first function will handle the interactions between our env and our agents.  
This function need to return the result of the run, so that we can visualize it later.

1. The second function will wrap the run function to be able to easily run experiments.  
This function need to return the mean of all the runs, so that we can visualize it later.

1. The third function will help us visualize the performance of our agent.

### Average performance of $\epsilon$-greedy action-value methods on the 10-armed testbed

In [ ]:
nb_exps = 2000

In [ ]:
agent_01 = EpsilonGreedy(nb_actions=env.action_space.n, epsilon=0.01)
mean_rewards_01, percent_optimal_action_01 = run_exp(nb_exps, env, agent_01)

In [ ]:
agent_1 = EpsilonGreedy(nb_actions=env.action_space.n, epsilon=0.1)
mean_rewards_1, percent_optimal_action_1 = run_exp(nb_exps, env, agent_1)

In [ ]:
agent_greedy = EpsilonGreedy(nb_actions=env.action_space.n, epsilon=0.0)
mean_rewards_0, percent_optimal_action_0 = run_exp(nb_exps, env, agent_greedy)

In [ ]:
results_exp_1 = {
    'Egreedy 0.01': {
        "mean_reward": mean_rewards_01,
        "optimal_action": percent_optimal_action_01,
        "color": "red"
    },
    'Egreedy 0.1': {
        "mean_reward": mean_rewards_1,
        "optimal_action": percent_optimal_action_1,
        "color": "blue"
    },
    'Greedy 0.0': {
        "mean_reward": mean_rewards_0,
        "optimal_action": percent_optimal_action_0,
        "color": "green"
    },
}

plot_results(
    "Compares greedy method with different parameters (0.01, 0.1 and 0)",
    ["Average Reward / Steps", "Optimal Action / Steps"],
    results_exp_1
)

Observations:

- Greedy Method: Initially improves faster but plateaus at a lower reward level (≈1), performing poorly long-term due to getting stuck on suboptimal actions. Finds the optimal action only about 33% of the time.

- Epsilon-Greedy (ε=0.1): Explores more, finds the optimal action earlier, but never selects it more than 91% of the time.

- Epsilon-Greedy (ε=0.01): Improves slower but eventually outperforms the ε=0.1 method in both reward and optimal action selection.
General: Epsilon-greedy methods outperform the greedy method due to continued exploration. Reducing epsilon over time can combine the benefits of high and low epsilon values.

## Incremental Implementation

The action-value methods we have discussed so far all estimate action values as sample
averages of observed rewards. We now turn to the question of how these averages can be
computed in a computationally efficient manner, in particular, with constant memory and constant per-time-step computation.

$$
\begin{aligned}
Q_n &= {R_1 + R_2 + ... + R_{n-1} \over n - 1} \\
\\
Q_{n + 1} &= {1 \over n} \sum{i=1}\\
       &= {\sum_{i=1}^{t-1} R_i · 1_{A_i = a} \over \sum_{i=1}^{t-1} 1_{A_i=a}}
\end{aligned}
$$

So, We can write Epsilon Greedy algorithm like this:

In [ ]:
@jitclass([
    ('nb_actions', int32),
    ('epsilon', float64),
    ('q', float64[:]),
    ('nb_action_taken', float64[:]),
])
class EpsilonGreedy():

    def __init__(self, nb_actions, epsilon):
        self.nb_actions = nb_actions
        self.epsilon = epsilon

        self.q = np.zeros(self.nb_actions)
        self.nb_action_taken = np.ones(self.nb_actions)

    def action(self):
        take_random_action_prob = np.random.uniform(0, 1)

        if take_random_action_prob < self.epsilon:
            return np.random.randint(0, self.nb_actions)
        else:
            return np.argmax(self.q)
    
    def observe(self, action, reward):
        self.nb_action_taken[action] += 1
        self.q[action] += (reward - self.q[action]) / self.nb_action_taken[action]
    
    def reset(self):
        self.q = np.zeros(self.nb_actions)
        self.nb_action_taken = np.ones(self.nb_actions)

2.5 Tracking a Nonstationary Problem

2.6 Optimistic Initial Values

In [ ]:
@jitclass([
    ('nb_actions', int32),
    ('epsilon', float64),
    ('optimistic_value', float64),
    ('q', float64[:]),
    ('nb_times_taken_action', float64[:]),
])
class EpsilonGreedyOptimisticInitValues():

    def __init__(self, nb_actions, epsilon=0.1, optimistic_value=0):
        self.nb_actions = nb_actions
        self.epsilon = epsilon
        self.optimistic_value = optimistic_value

        self.nb_times_taken_action = np.zeros(self.nb_actions)
        self.q = np.ones(self.nb_actions) * optimistic_value

    def action(self):
        take_random_action_prob = np.random.uniform(0, 1)

        if take_random_action_prob < self.epsilon:
            return np.random.randint(0, self.nb_actions)
        else:
            return np.argmax(self.q)
    
    def observe(self, action, reward):
        self.nb_times_taken_action[action] += 1
        self.q[action] += 0.1 * (reward - self.q[action]) # / self.nb_times_taken_action[action]
    
    def reset(self):
        self.nb_times_taken_action = np.zeros(self.nb_actions)
        self.q = np.ones(self.nb_actions) * self.optimistic_value


In [ ]:
agent_optimistic_greedy = EpsilonGreedyOptimisticInitValues(nb_actions=env.action_space.n, epsilon=0, optimistic_value=5)
mean_rewards_optimistic, percent_optimal_action_optimistic = run_exp(nb_exps, env, agent_optimistic_greedy)

In [ ]:
agent_non_optimistic = EpsilonGreedy(nb_actions=env.action_space.n, epsilon=0.1)
mean_rewards_non_optimistic, percent_optimal_action_non_optimistic = run_exp(nb_exps, env, agent_non_optimistic)

In [ ]:
results_exp_2 = {
    'Greedy Optimistic': {
        "mean_reward": mean_rewards_optimistic,
        "optimal_action": percent_optimal_action_optimistic,
        "color": "blue"
    },
    'Egreedy Non Optimistic': {
        "mean_reward": mean_rewards_non_optimistic,
        "optimal_action": percent_optimal_action_non_optimistic,
        "color": "red"
    }
}

plot_results("Optimistic greedy vs Non optimistic 0.01", ["Average Reward / Steps", "Optimal Action / Steps"], results_exp_2)

### Upper-Confidence-Bound Action Selection

In [ ]:
from numba import int32, float64
from numba.experimental import jitclass

@jitclass([
    ('nb_actions', int32),
    ('confidence', float64),
    ('alpha', float64),
    ('q', float64[:]),
    ('nb_action_taken', float64[:]),
    ('upper_configdence', float64[:]),
])
class UpperConfidenceBound():

    def __init__(self, nb_actions, confidence, alpha=0.1):
        self.nb_actions = nb_actions
        self.confidence = confidence
        self.alpha = alpha

        self.q = np.zeros(self.nb_actions)
        self.nb_action_taken = np.zeros(self.nb_actions)
        self.upper_configdence = np.ones(self.nb_actions) * np.inf

    def action(self):
        return np.argmax(self.q + self.upper_configdence)
    
    def observe(self, action, reward):
        self.nb_action_taken[action] += 1

        self.q[action] += (reward - self.q[action]) / self.nb_action_taken[action]

        if not 0 in self.nb_action_taken:
            self.upper_configdence = self.confidence * np.sqrt(np.log(np.sum(self.nb_action_taken)) / self.nb_action_taken)
        else:
            self.upper_configdence[action] = 0
    
    def reset(self):
        self.q = np.zeros(self.nb_actions)
        self.nb_action_taken = np.zeros(self.nb_actions)
        self.upper_configdence = np.ones(self.nb_actions) * np.inf


In [ ]:
nb_exps = 2000

In [ ]:
upper_condidence_agent = UpperConfidenceBound(nb_actions=env.action_space.n, confidence=2, alpha=0.1)
mean_rewards_upper_confidence, percent_optimal_action_upper_confidence = run_exp(nb_exps, env, upper_condidence_agent)
#1m8.1s vs 21s w numba


In [ ]:
egreedy_agent = EpsilonGreedy(nb_actions=env.action_space.n, epsilon=0.1)
mean_rewards_egreedy, percent_optimal_action_egreedy = run_exp(nb_exps, env, egreedy_agent)

In [ ]:
results_exp_3 = {
    'UCB': {
        "mean_reward": mean_rewards_upper_confidence,
        "optimal_action": percent_optimal_action_upper_confidence,
        "color": "blue"
    },
    'Egreedy 0.1': {
        "mean_reward": mean_rewards_egreedy,
        "optimal_action": percent_optimal_action_egreedy,
        "color": "red"
    }
}

plot_results("Upper Confidence Bound vs Epsilon Greedy", ["Average Reward / Steps", "Optimal Action / Steps"], results_exp_3)


2.8 Gradient Bandit Algorithms

In [ ]:
@jitclass([
    ('nb_actions', int32),
    ('alpha', float64),
    ('q', float64[:]),
    ('soft_probs', float64[:]),
    ('mean_reward', float64),
    ('nb_action_taken', float64),
])
class GradientBandit():

    def __init__(self, nb_actions, alpha):
        self.nb_actions = nb_actions
        self.alpha = alpha
        self.soft_probs = np.zeros(nb_actions)
        self.nb_action_taken = 0

        self.mean_reward = 0
        self.q = np.zeros(self.nb_actions)

    def _softmax(self):
        e_x = np.exp(self.q - np.max(self.q))
        probs = e_x / e_x.sum(axis=0)
        return probs
    
    def rand_choice_nb(self, prob):
        return np.searchsorted(np.cumsum(prob), np.random.random(), side="right")

    def action(self):
        self.soft_probs = self._softmax()
        return self.rand_choice_nb(self.soft_probs)
    
    def observe(self, action, reward):
        self.nb_action_taken += 1
        self.mean_reward += (reward - self.mean_reward) / self.nb_action_taken

        self.soft_probs[action] = - (1 - self.soft_probs[action])
        self.q -= self.alpha * (reward - self.mean_reward) * self.soft_probs
    
    def reset(self):
        self.nb_action_taken = 0
        self.q = np.zeros(self.nb_actions)


In [ ]:
nb_exps = 2000

In [ ]:
gb_agent = GradientBandit(nb_actions=env.action_space.n, alpha=0.1)
mean_rewards_gb, percent_optimal_action_gb = run_exp(nb_exps, env, gb_agent)
#2m23s vs 23s w numba

In [ ]:
egreedy_agent = EpsilonGreedy(nb_actions=env.action_space.n, epsilon=0.1)
mean_rewards_egreedy, percent_optimal_action_egreedy = run_exp(nb_exps, env, egreedy_agent)

In [ ]:
results_exp_4 = {
    'GradientBandit Alpha=0.1': {
        "mean_reward": mean_rewards_gb,
        "optimal_action": percent_optimal_action_gb,
        "color": "blue"
    },
    'Egreedy 0.01': {
        "mean_reward": mean_rewards_egreedy,
        "optimal_action": percent_optimal_action_egreedy,
        "color": "green"
    }
}

plot_results("Gradient Bandit vs Epsilon Greedy", ["Average Reward / Steps", "Optimal Action / Steps"], results_exp_4)

2.9 Associative Search (Contextual Bandits)

2.10 Summary

parameter study

In [ ]:
def run_parameter_study(nb_exps, env, agents_parameters):
    results_mean_reward = {}
    results_percent_optimal_action = {}

    for agent_name in agents_parameters:
        results_mean_reward[agent_name] = []
        results_percent_optimal_action[agent_name] = []

    for agent_name in agents_parameters:
        print(agent_name)

        for parameter in agents_parameters[agent_name]["parameters"]:
            print("    running parameters:", parameter)

            agent = agents_parameters[agent_name]["class"](nb_actions=env.action_space.n, **parameter)

            mean_reward_over_steps, percent_optimal_action_over_steps = run_exp(nb_exps, env, agent)

            mean_reward = np.mean(mean_reward_over_steps)
            mean_optimal_action_percent = np.mean(percent_optimal_action_over_steps)

            results_mean_reward[agent_name].append(mean_reward)
            results_percent_optimal_action[agent_name].append(mean_optimal_action_percent)

    return results_mean_reward, results_percent_optimal_action

In [ ]:
def plot_parameter_study_results(agents_parameters, results_mean_reward, results_percent_optimal_action):
    fig = make_subplots(rows=1, cols=2, subplot_titles=["Mean Reward / Parameters", "Mean Optimal Action / Parameters"])

    x = []

    for agent_name in results_mean_reward:
        parameter = agents_parameters[agent_name]["variable"]
        x += [p[parameter] for p in agents_parameters[agent_name]["parameters"]]

        fig.add_trace(
            go.Scatter(x=[p[parameter] for p in agents_parameters[agent_name]["parameters"]],
                       y=results_mean_reward[agent_name], line_color=agents_parameters[agent_name]["color"],
                       name=agent_name)
        , row=1, col=1)

        fig.add_trace(
            go.Scatter(x=[p[parameter] for p in agents_parameters[agent_name]["parameters"]],
                       y=results_percent_optimal_action[agent_name], line_color=agents_parameters[agent_name]["color"],
                       showlegend=False)
        , row=1, col=2)

    fig.update_layout(
        title="Parameter Study",
        legend_title="Parameters",
    )

    fig.update_xaxes(
        type='category',
        tickmode= 'array',
        categoryorder= 'array',
        categoryarray= sorted(x))

    fig.show()

In [ ]:
agents = {
    "EpsilonGreedy": {
        "class": EpsilonGreedy,
        "color": "red",
        "variable": "epsilon",
        "parameters": [
            {"epsilon": 1 / 128},
            {"epsilon": 1 / 64},
            {"epsilon": 1 / 32},
            {"epsilon": 1 / 16},
            {"epsilon": 1 / 8},
            {"epsilon": 1 / 4}
        ],
    },

    "Greedy Optimistic": {
        "class": EpsilonGreedyOptimisticInitValues,
        "color": "black",
        "variable": "optimistic_value",
        "parameters": [
            {"epsilon": 0, "optimistic_value": 1 / 4},
            {"epsilon": 0, "optimistic_value": 1 / 2},
            {"epsilon": 0, "optimistic_value": 1},
            {"epsilon": 0, "optimistic_value": 2},
            {"epsilon": 0, "optimistic_value": 4},
        ],
    },

    "UCB": {
        "class": UpperConfidenceBound,
        "color": "blue",
        "variable": "confidence",
        "parameters": [
            {"confidence": 1 / 16},
            {"confidence": 1 / 8},
            {"confidence": 1 / 4},
            {"confidence": 1 / 2},
            {"confidence": 1},
            {"confidence": 2},
            {"confidence": 4},
        ],
    },

    "Gradient Bandit": {
        "class": GradientBandit,
        "color": "green",
        "variable": "alpha",
        "parameters": [
            {"alpha": 1 / 32},
            {"alpha": 1 / 16},
            {"alpha": 1 / 8},
            {"alpha": 1 / 4},
            {"alpha": 1 / 2},
            {"alpha": 1},
            {"alpha": 2},
        ],
    }
}

In [ ]:
nb_exps = 2000
results_mean_reward, results_percent_optimal_action = run_parameter_study(nb_exps, env, agents)

In [ ]:
plot_parameter_study_results(agents, results_mean_reward, results_percent_optimal_action)